In [1]:
!nvidia-smi

Thu Mar 20 16:46:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.70                 Driver Version: 572.70         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   55C    P0             22W /  105W |    1546MiB /   8188MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#5.6.1. 计算设备
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [8]:
#5.6.2. 张量与GPU
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [9]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [13]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.1065, 0.8245, 0.6185],
        [0.7615, 0.6969, 0.4271]], device='cuda:0')

In [17]:
Z = x.cuda(0)
print(x)
print(Z)

tensor([1, 2, 3])
tensor([1, 2, 3], device='cuda:0')


In [18]:
Y + Z

tensor([[1.1065, 2.8245, 3.6185],
        [1.7615, 2.6969, 3.4271]], device='cuda:0')

In [19]:
Z.cuda(0) is Z

True

根据经验，多个小操作比一个大操作糟糕得多。 此外，一次执行几个操作比代码中散布的许多单个操作要好得多。 如果一个设备必须等待另一个设备才能执行其他操作， 那么这样的操作可能会阻塞。 这有点像排队订购咖啡，而不像通过电话预先订购： 当客人到店的时候，咖啡已经准备好了。

最后，当我们打印张量或将张量转换为NumPy格式时， 如果数据不在内存中，框架会首先将其复制到内存中， 这会导致额外的传输开销。 更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成。

In [20]:
#5.6.3. 神经网络与GPU
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [21]:
net(X)

tensor([[0.1679],
        [0.1679]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [22]:
net[0].weight.data.device

device(type='cuda', index=0)

In [28]:
#尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？
import torch
import time

# 大矩阵测试（2000x2000）
def test_big_matrix():
    # CPU计算
    a_cpu = torch.randn(10000, 10000)
    b_cpu = torch.randn(10000, 10000)
    
    start = time.time()
    torch.matmul(a_cpu, b_cpu)
    cpu_time = time.time() - start
    
    # GPU计算
    a_gpu = a_cpu.cuda()
    b_gpu = b_cpu.cuda()
    
    # 预热GPU
    torch.cuda.synchronize()
    start = time.time()
    torch.matmul(a_gpu, b_gpu)
    torch.cuda.synchronize()
    gpu_time = time.time() - start
    
    print(f"大矩阵计算：CPU: {cpu_time:.4f}s | GPU: {gpu_time:.4f}s")

# 小矩阵测试（10x10）    
def test_small_matrix():
    a = torch.randn(10, 10)
    b = torch.randn(10, 10)
    
    # CPU计算
    start = time.time()
    torch.matmul(a, b)
    cpu_time = time.time() - start
    
    # GPU计算
    a_gpu = a.cuda()
    b_gpu = b.cuda()
    
    torch.cuda.synchronize()
    start = time.time()
    torch.matmul(a_gpu, b_gpu)
    torch.cuda.synchronize()
    gpu_time = time.time() - start
    
    print(f"小矩阵计算：CPU: {cpu_time:.4f}s | GPU: {gpu_time:.4f}s")

test_big_matrix()
test_small_matrix()

大矩阵计算：CPU: 2.6155s | GPU: 0.2411s
小矩阵计算：CPU: 0.0000s | GPU: 0.0000s
